In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1gwaj_LpfX-bah9Cec-Run8TFBMxXaVvs/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1gwaj_LpfX-bah9Cec-Run8TFBMxXaVvs
To: /paraphrase_training_data.zip
100% 69.7M/69.7M [00:01<00:00, 37.4MB/s]


In [ ]:
!unzip paraphrase_training_data.zip -d 'data'

Archive:  paraphrase_training_data.zip
   creating: data/competition_data/
 extracting: data/competition_data/training.zip  
 extracting: data/competition_data/validation.zip  
  inflating: data/competition_data/evaluation.py  
  inflating: data/competition_data/training_eval.csv  


In [ ]:
%cd data
!unzip -q competition_data/training.zip -d "training_data"
!unzip -q competition_data/validation.zip -d "validation_data"

[Errno 2] No such file or directory: 'data'
/content/data


In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1uJQE3Z_gtAKWkE7gU8olSPuoo72nZyFD/view?usp=sharing
!unzip -q competition.zip -d 'data'
%cd data
!unzip -q training.zip -d "training_data"
!unzip -q validation_text.zip -d "validation_data"
# !unzip -q training/training_info.zip -d 'training_data/info'
# !unzip -q training_data/training_text.zip -d 'training_data/text'
# !unzip -q validation_data/validation_text.zip -d 'validation_data/validation_text'

Downloading...
From: https://drive.google.com/uc?id=1uJQE3Z_gtAKWkE7gU8olSPuoo72nZyFD
To: /content/competition.zip
100% 15.4M/15.4M [00:00<00:00, 27.7MB/s]
/content/data


In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/14NFaF2XbuData18WFJXXzwHZIzmr5W89/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=14NFaF2XbuData18WFJXXzwHZIzmr5W89
To: /content/data/testing_text.rar
100% 6.25M/6.25M [00:00<00:00, 43.0MB/s]


In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/12JzJZEMfRajtsP7Q0k2NP1vJelx4I0P_/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=12JzJZEMfRajtsP7Q0k2NP1vJelx4I0P_
To: /content/indices_training_validation.tar.gz
100% 12.7M/12.7M [00:00<00:00, 54.6MB/s]


In [ ]:
!tar -xf indices_training_validation.tar.gz

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 4.4 MB 14.7 MB/s 
     |████████████████████████████████| 1.2 MB 38.2 MB/s 
     |████████████████████████████████| 101 kB 11.4 MB/s 
     |████████████████████████████████| 596 kB 72.8 MB/s 
     |████████████████████████████████| 6.6 MB 56.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9961f90b50d15ddfecaedc26ac14351c36f6ea90fb404a95efbf1887166fd6d6
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cd ..

/content


In [ ]:
#@title Write BertBaseline.py
%%writefile BertBaseline.py 
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 30 21:30:43 2022

@author: jyotm

script for sentence transformer baseline
use python ..\evaluation.py --ans sentenceTransformer_baseline_results.csv --evl training_eval.csv
for local evaluation of training prediction
"""

import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import os
import sys
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from pprint import pprint
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import sentence_transformers
from subprocess import Popen

# nltk.download('punkt')
N =2000
# MODEL = 'paraphrase-albert-small-v2'
MODEL = 'all-MiniLM-L12-v2'
TRAIN = False


def read_info(file_name):
    xml_data = open(file_name,'r', encoding="utf8").read()
    root = ET.XML(xml_data)
    data = []
    for child in root:
        data.append(subchild.text for subchild in child)
    df = pd.DataFrame(data)
    df.columns = list(s.tag for s in root[0])
    return df

def create_dataset(root, training_files):
    dataset = {}
    #create dictionary from files
    print("loading training files.......")
    for file in tqdm(training_files):
        # print(file)
        raw_text = open(os.path.join(root,file),'r', encoding="utf8").read()
        get_sentences_with_ids(dataset,file,raw_text)
    return dataset

def get_sentences_with_ids(dataset_dct, fname,content):
    #create sent id first
    file_id = int(re.search(r"\d+", fname).group(0))
    paragraphs = content.split('\n')
    for i, para in enumerate(paragraphs):
        sentences = sent_tokenize(para)
        for j, sent in enumerate(sentences):
            if len(sent) < 20:
                continue
                # print("alert!!!!")
                # print("##"*30)
            dataset_dct[f'{file_id}_{i}_{j}'] = sent
    return dataset_dct

def create_id(file_id,para_id,sent_id):
    file_id = int(re.search(r"\d+", file_id).group(0))
    return f'{file_id}_{para_id}_{sent_id}'
    
def get_dataset_info(root,training_info):
    #get dataset from list of files
    dataset_info = {}
    cnt_none = 0
    print("loading training info...... ")
    for file in tqdm(training_info):
        df = read_info(os.path.join(root,file))
        #the df will contain file_names in columns if its empty
        if 'file_names' in df.columns:
            #skip that file in training_info
            df = None
            cnt_none += 1
        else:
            df['source_ids'] = df.apply(lambda x: create_id(x.file_ids, x.para_ids,x.sens_ids), axis=1)
            df['target_ids'] = df.apply(lambda x: create_id(x.tag_file_ids, x.tag_para_ids,x.tag_sens_ids), axis=1)
        dataset_info[file.split(".")[0]] = df
    print(f"Total no of non pleg files in dataset is: {(len(training_info)-cnt_none)} and percentage is: {(len(training_info)-cnt_none)/len(training_info)}")
    return dataset_info

def get_vector_encodings(dataset,model):
    vectors = {}
    # for key in tqdm(dataset.keys()):
    #     vectors[key] = model.encode([dataset[key]])
    #more efficient way
    vectors = model.encode(list(dataset.values()),show_progress_bar=True)
    vectors = {i:j for i,j in zip(dataset.keys(),vectors)}
    return vectors

def get_solution_parapharse_mining(model,dataset):
    #iterate over dataset to  find the cosine distances
    rev_subset = {}
    for a,b in dataset.items():
        rev_subset[b] = a
    #use inbuilt function for paraphrase mining
    solution = sentence_transformers.util.paraphrase_mining(model,list(dataset.values()),query_chunk_size=10000,corpus_chunk_size=60000,show_progress_bar=True, top_k = 100)
    return (solution,rev_subset)
    
def get_solution_pairs(sub_dataset,dataset_info):
    pairs = {}
    source_text_ids = sub_dataset.keys()
    for k in dataset_info.keys():
        df = dataset_info[k]
        if df is not None:
            source_ids = df.source_ids.values
            target_ids = df.target_ids.values
            for x,y in zip(source_ids,target_ids):
                if x in source_text_ids:
                    pairs[x] = y
    return pairs
            
        

def get_subset_dataset(training_files,dataset_info,n=100):
    #only get subset of dataset with n files and their correspodning target files
    corresponding_target_files = {}
    subset_dataset = {}
    for file in dataset_info.keys():
        if dataset_info[file] is not None:
            corresponding_target_files[file] = dataset_info[file]['tag_file_ids'].unique()
        else:
            #the file does not contain pleg and hence the info is empty
            continue
    subset_dataset = {}
    root = 'training_text' if TRAIN else 'validation_text'
    for fname in tqdm(training_files[:n]):
        raw_text = open(os.path.join(root,fname),'r', encoding="utf8").read()
        get_sentences_with_ids(subset_dataset,fname,raw_text)
        #check if the file does contain pleg (or hence occurs in corresponding target files keys)
        key = fname.split('.')[0]
        if key in corresponding_target_files.keys():
            target_files = corresponding_target_files[key]
            for ftg in target_files:
                raw_txt_tg = open(os.path.join(root,ftg + ".newtext"),'r', encoding="utf8").read()
                get_sentences_with_ids(subset_dataset,ftg,raw_txt_tg)
    print("subset dataset created successfully.........")
    return subset_dataset

if __name__ == '__main__':
    subroot = '\\training' if TRAIN else '\\validation'
    os.chdir(r'C:\Users\jyotm\Documents\Data Science Competitions\TextParaphraseDetection\paraphrase\CRI-Comp-2022-Text-Paraphrase-Detection-Challenge\competition_data'+subroot)
    if TRAIN:
        fname = 'training_info\gen_doc0.newinfo'
        print(os.path.isfile(fname))
        df = read_info(fname)
        print(df.columns, df.shape)
        for i in df.columns:
            print(df[i])
        # print(df['tag_plag_sens'][0])
        training_files = os.listdir('training_text')
        training_info = os.listdir('training_info')
        df['source_ids'] = df.apply(lambda x: create_id(x.file_ids, x.para_ids,x.sens_ids), axis=1)
        print(df['source_ids'])

    model = SentenceTransformer(MODEL,device='cuda')
    root = 'training_text' if TRAIN else 'validation_text'
    if TRAIN:
        dataset_info = get_dataset_info('training_info',training_info)
        # sub_dataset = get_subset_dataset(training_files, dataset_info, n = N)
        sub_dataset = create_dataset(root,training_files)
    else:
        #for validation, we don't have info files
        val_files = os.listdir('validation_text')
        sub_dataset = create_dataset(root,val_files)

    # embeddings = get_vector_encodings(sub_dataset,model)
    # print(embeddings['0_1_0'])
    
    #get solution for subseet
    soln, rev_ds = get_solution_parapharse_mining(model,sub_dataset)
    
    print("Paraphrase extracted...........")
    # thresholds = [0.9,0.8,0.7,0.6,0.5]
    thresholds = [0.81,0.82,0.83,0.84,0.85,0.86,0.87,0.88,0.89,0.95]
    
    for th in thresholds:
        key_ind_subset = list(sub_dataset.keys())
        soln_pairs = {}
        for (score, id1,id2) in soln:
            if score > th and key_ind_subset[id1] not in soln_pairs.keys():
                soln_pairs[key_ind_subset[id1]] = key_ind_subset[id2]
        
        save_preds = True
        if save_preds:
            val_df = pd.DataFrame(soln_pairs.items())
            val_df.columns = ['id1','id2']
            df_name = f"..\Val_sentenceTransformer_baseline_results_thres_{th}.csv"
            df_name = df_name.replace('Val','train') if TRAIN else df_name
            val_df.to_csv(df_name,index=False)
        
        if TRAIN:
            gt_pairs = get_solution_pairs(sub_dataset, dataset_info)
            cnt = 0
            total = len(gt_pairs)
            for sol in soln_pairs.keys():
                if sol in gt_pairs.keys():
                    if soln_pairs[sol] == gt_pairs[sol]:
                        cnt += 1
            
            print(f"The accuracy of this method on subset is: {cnt/total}")
            
            
    
    # print(dataset)
    print("#"*20)
    #verify the dataset creation by keys from info file
    # ids = ['0_'+str(i)+'_'+str(j) for i,j in zip(list(df.para_ids.values),list(df.sens_ids.values))]
    # print(ids)
    # print([dataset[sen] for sen in ids])
    # print(df.plag_sens.values)
    # print("#"*20)
    # print([dataset[s] for s in dataset.keys() if s[2:3] == '0'])
    # print(dataset.keys())
    # print(dataset['0_1_0'])
    
    
    
'''

Results:
    on training set: (The scores are calculated using evaluation.py script against training_eval.csv )
        subset size | Model | recall | f1 score | precision | other hyper parameter imp change (time to calculate batches embeddings)
        1000 | 'all-MiniLM-L12-v2' | 0.02 | 0.01 | 0.01 | with corpus_chunk_size = 1k and top_k = 2 (something went wrong here, dk)
        
        1000 | 'all-MiniLM-L12-v2' | 0.289 | 0.058 | 0.032 | with corpus_chunk_size = 10k and top_k = 2  and threshold = 0.5 (time: 3:11)
        1000 | 'all-MiniLM-L12-v2' | 0.289 | 0.061 | 0.035 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.5 (Time: 2:53)
        1000 | 'all-MiniLM-L12-v2' | 0.198 | 0.198 | 0.198 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.9 (Time: 2:53)
        1000 | 'all-MiniLM-L12-v2' | 0.261 | 0.144 | 0.185 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.8 (Time: 2:53)
        1000 | 'all-MiniLM-L12-v2' | 0.283 | 0.057 | 0.096 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.7 (Time: 2:53)
        following are the score with above model but with different thresholds (0.81 to 0.89)
        Thres: 0.81 | 0.82 | 0.83 | 0.84 | 0.85 | 0.86 | 0.87 | 0.88 | 0.89
        Recall: 0.255 | 0.251 | 0.247 | 0.241 | 0.237 | 0.230 | 0.223 | 0.214 | 0.206
        Precision: 0.153| 0.162 | 0.171 | 0.179 | 0.187 | 0.193 | 0.197 | 0.198 | 0.198
        F1-score: 0.192| 0.197 | 0.202 | 0.205 | 0.2094 | 0.2097 | 0.2094 | 0.206 | 0.202
        
        Conclusion: the highest f1 score for subset 1000 is from threshold 0.86
                    the highest f1 score for subset 2000 occurs at threshold 0.87 (The scores for 0.86 & 0.87 is 0.2355 vs 0.2363)
                    for the full dataset highest f1score occurs at thres 0.87 with value 0.2348 (towards 0.88)        
                
        1000 | 'all-MiniLM-L12-v2' | 0.289 | 0.062 | 0.034 | with corpus_chunk_size = 20k and top_k = 50 and threshold = 0.5 (Time: 3:00)
        2000 | 'all-MiniLM-L12-v2' | 0.502 | 0.050 | 0.091 | with corpus_chunk_size = 20k and top_k = 100 and threshold = 0.5 (Time: 5:06)
        full dataset | 'all-MiniLM-L12-v2' | 0.653 | 0.059 | 0.109 | with corpus_chunk_size = 20k and top_k = 100 and threshold = 0.5 (Time: 6:45 ) + misc time: 4:30
        
        1000 | 'paraphrase-albert-small-v2' | 0.29 | 0.067 | 0.04 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.5 (Time: 6:25)
        
        
'''

Writing BertBaseline.py


In [ ]:
import BertBaseline
import importlib
import numpy as np
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
importlib.reload(BertBaseline)

root = '/content/data/training_data/training/info'
training_info = os.listdir(root)
dataset_info = BertBaseline.get_dataset_info(root,training_info)

def get_input_examples(dataset):
    sentence_dataset = []
    for key in dataset.keys():
        if dataset[key] is not None:
            for df in dataset[key].itertuples():
                score = np.random.uniform(low=0.93, high=1.0)
                # sent = dataset[key]['plag_sens'].values[0]
                # target = dataset[key]['tag_plag_sens'].values[0]
                sent = df[5]
                target = df[9]
                input_sample = InputExample(texts=[sent,target], label=score)
                sentence_dataset.append(input_sample)
    return sentence_dataset
dataset = get_input_examples(dataset_info)

# training_dataset,validation_dataset = train_test_split(dataset, test_size=0.1,random_state=42)
training_dataset = dataset
validation_dataset = dataset[-100:]

loading training info...... 


100%|██████████| 4000/4000 [00:08<00:00, 470.19it/s]


Total no of non pleg files in dataset is: 2746 and percentage is: 0.6865


In [ ]:
x = pd.read_csv('/content/data/training_eval_new.csv')
print(len(training_dataset))
x.describe()

6212


,id1,id2
count,6212,6212
unique,6212,6212
top,1_0_3,36_0_3
freq,1,1


In [ ]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

# Read the dataset
# model_name = 'cross-encoder/stsb-roberta-base'
# model_name = 'all-roberta-large-v1'
# model_name = 'all-mpnet-base-v1'
model_name = 'paraphrase-multilingual-mpnet-base-v2'
train_batch_size = 16
num_epochs = 2
model_save_path = 'output/training_paraphrase_finetuning_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name)

# Convert the dataset to a DataLoader ready for training
logging.info("Setting up dataloader.....")

train_dataloader = DataLoader(training_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

# Development set: Measure correlation between cosine score and gold labels
logging.info("Setting up dataloader dev dataset.....")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(validation_dataset, name='para-dev')

# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))



2022-06-29 16:47:15 - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

2022-06-29 16:47:48 - Use pytorch device: cuda
2022-06-29 16:47:48 - Setting up dataloader.....
2022-06-29 16:47:48 - Setting up dataloader dev dataset.....
2022-06-29 16:47:48 - Warmup-steps: 78


In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=200,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/389 [00:00<?, ?it/s]

2022-06-29 16:48:55 - EmbeddingSimilarityEvaluator: Evaluating the model on para-dev dataset in epoch 0 after 200 steps:
2022-06-29 16:48:55 - Cosine-Similarity :	Pearson: 0.0745	Spearman: 0.1224
2022-06-29 16:48:55 - Manhattan-Distance:	Pearson: 0.0933	Spearman: 0.1196
2022-06-29 16:48:55 - Euclidean-Distance:	Pearson: 0.0941	Spearman: 0.1230
2022-06-29 16:48:55 - Dot-Product-Similarity:	Pearson: 0.1257	Spearman: 0.1246
2022-06-29 16:48:55 - Save model to output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-06-29_16-47-15
2022-06-29 16:49:48 - EmbeddingSimilarityEvaluator: Evaluating the model on para-dev dataset after epoch 0:
2022-06-29 16:49:48 - Cosine-Similarity :	Pearson: 0.0606	Spearman: 0.1177
2022-06-29 16:49:48 - Manhattan-Distance:	Pearson: 0.0921	Spearman: 0.1182
2022-06-29 16:49:48 - Euclidean-Distance:	Pearson: 0.0936	Spearman: 0.1249
2022-06-29 16:49:48 - Dot-Product-Similarity:	Pearson: 0.0329	Spearman: 0.0481
2022-06-29 16:49:48 - 

Iteration:   0%|          | 0/389 [00:00<?, ?it/s]

2022-06-29 16:50:43 - EmbeddingSimilarityEvaluator: Evaluating the model on para-dev dataset in epoch 1 after 200 steps:
2022-06-29 16:50:43 - Cosine-Similarity :	Pearson: 0.0642	Spearman: 0.1139
2022-06-29 16:50:43 - Manhattan-Distance:	Pearson: 0.0953	Spearman: 0.1076
2022-06-29 16:50:43 - Euclidean-Distance:	Pearson: 0.0978	Spearman: 0.1117
2022-06-29 16:50:43 - Dot-Product-Similarity:	Pearson: 0.0297	Spearman: 0.0391
2022-06-29 16:51:29 - EmbeddingSimilarityEvaluator: Evaluating the model on para-dev dataset after epoch 1:
2022-06-29 16:51:29 - Cosine-Similarity :	Pearson: 0.0755	Spearman: 0.1125
2022-06-29 16:51:29 - Manhattan-Distance:	Pearson: 0.1083	Spearman: 0.1199
2022-06-29 16:51:29 - Euclidean-Distance:	Pearson: 0.1109	Spearman: 0.1199
2022-06-29 16:51:29 - Dot-Product-Similarity:	Pearson: 0.0349	Spearman: 0.0464


In [ ]:


# test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(write_here, name='sts-test')
# test_evaluator(model, output_path=model_save_path)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r '/content/output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-06-29_06-24-19' '/content/gdrive/MyDrive/Paraphrase/fine_tuned_paraphrase-multilingual-mpnet-base-v2-with-max-score'

In [ ]:
!cp -r '/content/output/training_paraphrase_finetuning_training-all-mpnet-base-v2-2022-06-15_17-50-34' '/content/gdrive/MyDrive/Paraphrase detection exp/training_paraphrase_finetuning_training-all-mpnet-base-v2-2022-06-15_17-50-34' 

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
model_save_path

'output/training_paraphrase_finetuning_training-all-mpnet-base-v1-2022-06-25_03-13-18'

In [ ]:

importlib.reload(BertBaseline)
root = '/content/data/validation_data/validation_text'

val_dataset = BertBaseline.create_dataset(root,os.listdir(root))
len(val_dataset)

loading training files.......


100%|██████████| 3000/3000 [00:02<00:00, 1078.38it/s]


95845

In [ ]:
importlib.reload(BertBaseline)

root_info = '/content/data/training_data/info'
root_text = '/content/data/training_data/text'

info = BertBaseline.get_dataset_info(root_info,os.listdir(root_info))

subset = BertBaseline.get_subset_dataset(os.listdir(root_text),info,n=10)
len(subset)

loading training info...... 


100%|██████████| 4000/4000 [00:10<00:00, 373.35it/s]


Total no of non pleg files in dataset is: 2294 and percentage is: 0.5735


100%|██████████| 10/10 [00:00<00:00, 121.34it/s]

subset dataset created successfully.........


2566

In [ ]:
sens = list(subset.values())
from itertools import permutations

perms = list(permutations(sens, 2))
len(perms)

6581790

In [ ]:
# from sentence_transformers import CrossEncoder
# cross_enc = CrossEncoder('cross-encoder/stsb-roberta-base')
# predictions = cross_enc.predict(perms)


In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1uJQE3Z_gtAKWkE7gU8olSPuoo72nZyFD/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1uJQE3Z_gtAKWkE7gU8olSPuoo72nZyFD
To: /content/competition.zip
100% 15.4M/15.4M [00:00<00:00, 67.5MB/s]


In [ ]:
!unzip competition.zip -d 'val_data'
%cd val_data
!unzip -q validation_text.zip -d "validation_data"

Archive:  competition.zip
  inflating: val_data/evaluation.py  
  inflating: val_data/training.zip   
  inflating: val_data/training_eval_new.csv  
  inflating: val_data/validation_text.zip  
/content/val_data


In [ ]:
%cd ..

/content


In [ ]:
# model_save_path = 'all-roberta-large-v1'
# model_save_path = '/content/output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-06-29_06-24-19'

model = SentenceTransformer(model_save_path)

sol, _ = BertBaseline.get_solution_parapharse_mining(model,val_dataset)


2022-06-29 16:51:34 - Load pretrained SentenceTransformer: output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-06-29_16-47-15
2022-06-29 16:51:37 - Use pytorch device: cuda


Batches:   0%|          | 0/2996 [00:00<?, ?it/s]

In [ ]:
len(val_dataset)

95845

## Prediction analysis

In [ ]:
val_sentences = {}
for filename in tqdm(os.listdir('/content/data/validation_data/validation_text')):
    ind_filename = filename.replace('newtext','xml')
    doc_number = ind_filename.split('.')[0][7:]
    df = BertBaseline.read_info(f'/content/indices/{ind_filename}')
    df['indices'] = df['indices'].apply(pd.eval)
    for index, row in df.iterrows():
        sens_id = "_".join([doc_number, str(row['indices'][0]), str(row['indices'][1])])
        # print(doc_number, filename, row['indices'], sens_id)
        val_sentences[sens_id] = row['sentences']
len(val_sentences)

  0%|          | 0/3000 [00:00<?, ?it/s]

98172

In [ ]:
model = SentenceTransformer("/content/output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-06-29_16-47-15")
sol, _ = BertBaseline.get_solution_parapharse_mining(model,val_sentences)


Batches:   0%|          | 0/3068 [00:00<?, ?it/s]

In [ ]:
key_ind_subset = list(val_dataset.keys())
print(len(key_ind_subset))
soln_pairs = {}
scores_key = {}
for (score, id1,id2) in sol:
    if score > 0.96:
        if key_ind_subset[id1] not in soln_pairs.keys():
            soln_pairs[key_ind_subset[id1]] = [key_ind_subset[id2]]
            scores_key[key_ind_subset[id1]] = [score]
        else:
            soln_pairs[key_ind_subset[id1]].append(key_ind_subset[id2])
            scores_key[key_ind_subset[id1]].append(score)   
# scores_key['5421_2_0']

95845


In [ ]:
# #I noticed doing argmax of scores_key and found that all have first element as max score, so I choose that pair with solution
# # filtered_pairs = {}
# # for k in soln_pairs.keys():
# #     filtered_pairs[k] = soln_pairs[k][0]
# # val_df = pd.DataFrame(filtered_pairs.items())
# # val_df.columns = ['id1','id2']
# # df_name = f"filtered_results.csv"
# # val_df.to_csv(df_name,index=False)

# key_ind_subset = list(val_dataset.keys())
# print(len(key_ind_subset))
# soln_pairs = {}
# scores_key = {}
# for (score, id1,id2) in sol:
#     if 1.0 > score > 0.97:
#         if key_ind_subset[id1] not in soln_pairs.keys():
#             soln_pairs[key_ind_subset[id1]] = [key_ind_subset[id2]]
#             scores_key[key_ind_subset[id1]] = [score]
#         else:
#             soln_pairs[key_ind_subset[id1]].append(key_ind_subset[id2])
#             scores_key[key_ind_subset[id1]].append(score)  
# filtered_pairs = {}
# for k in soln_pairs.keys():
#     filtered_pairs[k] = soln_pairs[k][0]

# lst = []
# new_pairs = {}
# for k in filtered_pairs.keys():
#     id2 = filtered_pairs[k]
#     if id2 in filtered_pairs.keys():
#         if 1.0 > scores_key[id2][0] > scores_key[k][0]:
#             new_pairs[id2] = filtered_pairs[id2]
#             lst.append(id2)
#         else:
#             new_pairs[k] = id2
#     else:
#         new_pairs[k] = id2

# val_df = pd.DataFrame(new_pairs.items())
# val_df.columns = ['id1','id2']
# df_name = f"filtered_with_id2_results.csv"
# val_df.to_csv(df_name,index=False)

# len(new_pairs), len(filtered_pairs), len(lst)

95845


(4862, 5287, 468)

In [ ]:
pred = pd.read_csv('/content/output/cross_enc_new_result_0.66.csv')
gt = pd.read_csv('/content/validation_eval_new.csv')

z = 0
for x, (id1,id2) in gt.iterrows():
    print(id1, val_sentences[id1])
    print(id2, val_sentences[id2])
    print("###"* 10)
    z += 1
    if z > 10: 
        break

4002_1_3 One meta-analysis found weaker evidence for TGF-1 869C/T polymorphism and breast cancer risk, while the other meta-analysis looked at the association between 869C/T polymorphisms and HNC.
5670_5_0 One meta-analysis found weaker evidence for TGF-1 869C/T polymorphism and breast cancer risk, while the other meta-analysis looked at the association between 869C/T and HNC.
##############################
4002_4_1 Figure 3 shows the statistical significance of differentially expressed circRNAs between cases and controls.
6973_0_2 A volcano plot shows the statistical significance of differentially expressed circRNAs between cases and controls.
##############################
4004_1_1 The youngest subject in the study was 4 months old.
5889_2_2 The youngest subject in our study population was 4 months old.
##############################
4004_5_5 Hummel et al.
4572_3_4 Hummel et al. were in 1997.
##############################
4005_0_2 It is possible that high attentional impulsivity is 

In [ ]:
!python /content/evaluation.py --ans /content/filtered_with_id2_results.csv --evl /content/validation_eval_new.csv

100% 4862/4862 [00:02<00:00, 1795.18it/s]
100% 4862/4862 [00:00<00:00, 6131.01it/s]
Recall: 0.6389270976616231
Precision: 0.3821472645002057
F1-score: 0.4782496782496783


In [ ]:
ind = 119
print(key_ind_subset[sol[ind][1]], val_dataset[key_ind_subset[sol[ind][1]]])
print(key_ind_subset[sol[ind][2]], val_dataset[key_ind_subset[sol[ind][2]]])

4900_1_3 As many as 20% of patients with positive serology have no agreement on how to monitor their progress, despite the fact that such patients are increasing in number.
5666_1_1 As many as 20% of patients with positive serology have no agreement on how to monitor their progress, despite the fact that such patients are increasing in number.


In [ ]:
val_soln = pd.read_csv('/content/validation_eval_new.csv')
# val_soln[val_soln['id1'] == '4900_1_3']
z = 0
# for x, (id1,id2) in val_soln.iterrows():
    # print(id1, val_dataset[id1])
    # print(id2, val_dataset[id2])
    # print("###"* 10)
    # z += 1
    # if z > 10: 
    #     break

ids1 = val_soln.id1.unique().tolist()
ids2 = val_soln.id2.unique().tolist()
s1 = set(ids1) 
s2 = set(ids2)
print(len(ids1), len(ids2), len(s1), len(s2))
len(s2-set(val_dataset.keys())), s1.intersection(s2), len(s2-set(val_sentences.keys()))

2908 2908 2908 2908


(91, set())

In [ ]:
soln_pairs['4004_1_1'], val_dataset['6295_3_1']

(['6295_3_1'], 'Around 4.6% of sample infants were premature.')

In [ ]:
!zip -r finetuned_mpnet.zip /content/output/training_paraphrase_finetuning_training-all-mpnet-base-v1-2022-06-25_03-13-18 

  adding: content/output/training_paraphrase_finetuning_training-all-mpnet-base-v1-2022-06-25_03-13-18/ (stored 0%)
  adding: content/output/training_paraphrase_finetuning_training-all-mpnet-base-v1-2022-06-25_03-13-18/modules.json (deflated 62%)
  adding: content/output/training_paraphrase_finetuning_training-all-mpnet-base-v1-2022-06-25_03-13-18/sentence_bert_config.json (deflated 4%)
  adding: content/output/training_paraphrase_finetuning_training-all-mpnet-base-v1-2022-06-25_03-13-18/config_sentence_transformers.json (deflated 27%)
  adding: content/output/training_paraphrase_finetuning_training-all-mpnet-base-v1-2022-06-25_03-13-18/special_tokens_map.json (deflated 50%)
  adding: content/output/training_paraphrase_finetuning_training-all-mpnet-base-v1-2022-06-25_03-13-18/eval/ (stored 0%)
  adding: content/output/training_paraphrase_finetuning_training-all-mpnet-base-v1-2022-06-25_03-13-18/eval/similarity_evaluation_para-dev_results.csv (deflated 63%)
  adding: content/output/trai

In [ ]:
import pandas as pd

thresholds = [0.5, 0.81,0.88,0.94,0.95, 0.96, 0.97, 0.963,0.965,0.967, 0.98, 0.975, 0.99, 0.972,0.973,0.976]
    
# thresholds = [0.965, 0.966, 0.967, 0.968, 0.969, 0.9695, 0.97, 0.9705, 0.971, 0.9715, 0.972, 0.973]
for th in thresholds:
    key_ind_subset = list(val_dataset.keys())
    soln_pairs = {}
    for (score, id1,id2) in sol:
        if 1.0 > score > th and key_ind_subset[id1] not in soln_pairs.keys():
            soln_pairs[key_ind_subset[id1]] = key_ind_subset[id2]
    
    save_preds = True
    if save_preds:
        val_df = pd.DataFrame(soln_pairs.items())
        val_df.columns = ['id1','id2']
        df_name = f"output/Val_results_old_method_thres_{th}.csv"
        val_df.to_csv(df_name,index=False)

In [ ]:
res_thresholds = [0.94,0.95, 0.96, 0.97, 0.963,0.965,0.967, 0.98, 0.975, 0.99, 0.972,0.973,0.976]
for th in res_thresholds:
    !echo "predicting for {th}"
    !python /content/evaluation.py --ans '/content/output/Val_results_old_method_thres_{th}.csv' --evl /content/validation_eval_new.csv

predicting for 0.94
100% 26265/26265 [00:31<00:00, 839.43it/s]
100% 26265/26265 [00:04<00:00, 6243.71it/s]
Recall: 0.797799174690509
Precision: 0.08833047782219684
F1-score: 0.159051177458609
predicting for 0.95
100% 12664/12664 [00:06<00:00, 1811.12it/s]
100% 12664/12664 [00:01<00:00, 6516.11it/s]
Recall: 0.7792297111416782
Precision: 0.17893240682248895
F1-score: 0.2910351913691241
predicting for 0.96
100% 6688/6688 [00:01<00:00, 3437.07it/s]
100% 6688/6688 [00:00<00:00, 7093.39it/s]
Recall: 0.7379642365887208
Precision: 0.3208732057416268
F1-score: 0.44726969570654446
predicting for 0.97
100% 4292/4292 [00:00<00:00, 5294.59it/s]
100% 4292/4292 [00:00<00:00, 7542.09it/s]
Recall: 0.670907840440165
Precision: 0.4545666356011184
F1-score: 0.5419444444444445
predicting for 0.963
100% 5771/5771 [00:01<00:00, 3940.44it/s]
100% 5771/5771 [00:00<00:00, 7198.89it/s]
Recall: 0.7224896836313618
Precision: 0.36406168774909026
F1-score: 0.48415716096324457
predicting for 0.965
100% 5239/5239 [00:

## THIS IS EPIC, MY CODE HAD A BUG (one major bug, another was minor)

> Indented block



With using val_sentences extracted from indices, the solution on trained model is working really great, 

In [ ]:
res_thresholds = [0.94,0.95, 0.96, 0.97, 0.963,0.965,0.967, 0.98, 0.975, 0.99, 0.972,0.973,0.976]
for th in res_thresholds:
    !echo "predicting for {th}"
    !python /content/evaluation.py --ans '/content/output/Val_results_old_method_thres_{th}.csv' --evl /content/validation_eval_new.csv

predicting for 0.94
100% 27818/27818 [00:34<00:00, 807.95it/s]
100% 27818/27818 [00:04<00:00, 6288.81it/s]
Recall: 0.9566712517193948
Precision: 0.10000718958947444
F1-score: 0.18108442361517932
predicting for 0.95
100% 13705/13705 [00:08<00:00, 1669.61it/s]
100% 13705/13705 [00:02<00:00, 6576.13it/s]
Recall: 0.9401650618982118
Precision: 0.19948923750456038
F1-score: 0.32913983025341603
predicting for 0.96
100% 7434/7434 [00:02<00:00, 3060.48it/s]
100% 7434/7434 [00:01<00:00, 5056.04it/s]
Recall: 0.8951169188445667
Precision: 0.3501479687920366
F1-score: 0.5033842583639528
predicting for 0.97
100% 4784/4784 [00:00<00:00, 4830.93it/s]
100% 4784/4784 [00:00<00:00, 7852.37it/s]
Recall: 0.8170563961485557
Precision: 0.49665551839464883
F1-score: 0.6177847113884556
predicting for 0.963
100% 6438/6438 [00:01<00:00, 3515.25it/s]
100% 6438/6438 [00:00<00:00, 7455.73it/s]
Recall: 0.8772352132049519
Precision: 0.39624106865486175
F1-score: 0.5459019901562165
predicting for 0.965
100% 5840/5840 

check id2 is also in keys of soln, if yes, check for greater score (cosine sim)

In [ ]:
# thresholds = [0.965, 0.966, 0.967, 0.968, 0.969, 0.9695, 0.97, 0.9705, 0.971, 0.9715, 0.972, 0.973, 0.975, 0.979, 0.981, 0.985, 0.99, 0.995]
thresholds = [0.5, 0.7, 0.81,0.85,0.90, 0.94,0.95, 0.953, 0.957, 0.96, 0.97, 0.963,0.965,0.967, 0.98, 0.975, 0.99, 0.972,0.973,0.976]
for th in thresholds:
    key_ind_subset = list(val_dataset.keys())
    soln_pairs = {}
    scores_key = {}
    for (score, id1,id2) in sol:
        if 1.0 > score > th:
            if key_ind_subset[id1] not in soln_pairs.keys():
                soln_pairs[key_ind_subset[id1]] = [key_ind_subset[id2]]
                scores_key[key_ind_subset[id1]] = [score]
            else:
                soln_pairs[key_ind_subset[id1]].append(key_ind_subset[id2])
                scores_key[key_ind_subset[id1]].append(score)  
    filtered_pairs = {}
    for k in soln_pairs.keys():
        filtered_pairs[k] = soln_pairs[k][0]

    lst = []
    new_pairs = {}
    for k in filtered_pairs.keys():
        id2 = filtered_pairs[k]
        if id2 in filtered_pairs.keys():
            if 1.0 > scores_key[id2][0] > scores_key[k][0]:
                new_pairs[id2] = filtered_pairs[id2]
                lst.append(id2)
            else:
                new_pairs[k] = id2
        else:
            new_pairs[k] = id2

    val_df = pd.DataFrame(new_pairs.items())
    val_df.columns = ['id1','id2']
    df_name = f"output/filtered_with_mpnet_v1_results_{th}.csv"
    val_df.to_csv(df_name,index=False)

In [ ]:
for th in thresholds:
    !echo "predicting for {th}"
    !python /content/evaluation.py --ans '/content/output/filtered_with_mpnet_v1_results_{th}.csv' --evl /content/validation_eval_new.csv

predicting for 0.5
100% 17110/17110 [00:12<00:00, 1347.23it/s]
100% 17110/17110 [00:02<00:00, 6480.98it/s]
Recall: 0.7929848693259972
Precision: 0.1347749853886616
F1-score: 0.23039264661804373
predicting for 0.7
100% 17110/17110 [00:13<00:00, 1298.50it/s]
100% 17110/17110 [00:02<00:00, 5873.34it/s]
Recall: 0.7929848693259972
Precision: 0.1347749853886616
F1-score: 0.23039264661804373
predicting for 0.81
100% 17110/17110 [00:12<00:00, 1318.27it/s]
100% 17110/17110 [00:02<00:00, 6455.48it/s]
Recall: 0.7929848693259972
Precision: 0.1347749853886616
F1-score: 0.23039264661804373
predicting for 0.85
100% 17110/17110 [00:13<00:00, 1239.43it/s]
100% 17110/17110 [00:02<00:00, 6442.17it/s]
Recall: 0.7929848693259972
Precision: 0.1347749853886616
F1-score: 0.23039264661804373
predicting for 0.9
100% 17110/17110 [00:12<00:00, 1354.58it/s]
100% 17110/17110 [00:02<00:00, 6475.35it/s]
Recall: 0.7929848693259972
Precision: 0.1347749853886616
F1-score: 0.23039264661804373
predicting for 0.94
100% 102

In [ ]:
from sentence_transformers import CrossEncoder
cross_enc = CrossEncoder('cross-encoder/stsb-roberta-large')

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/139 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

# Read the dataset
model_name = 'cross-encoder/stsb-roberta-base'
# model_name = 'all-roberta-large-v1'
# model_name = 'all-mpnet-base-v1'
# model_name = 'paraphrase-multilingual-mpnet-base-v2'
train_batch_size = 16
num_epochs = 1
model_save_path = 'output/training_paraphrase_finetuning_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


# Load a pre-trained sentence transformer model
model = CrossEncoder(model_name)

# Convert the dataset to a DataLoader ready for training
logging.info("Setting up dataloader.....")

train_dataloader = DataLoader(training_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

# Development set: Measure correlation between cosine score and gold labels
logging.info("Setting up dataloader dev dataset.....")
evaluator = CECorrelationEvaluator.from_input_examples(validation_dataset, name='sts-dev')

# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


# Train the model
model.fit(train_dataloader = train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=200,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


2022-06-29 04:40:16 - Use pytorch device: cuda
2022-06-29 04:40:16 - Setting up dataloader.....
2022-06-29 04:40:16 - Setting up dataloader dev dataset.....
2022-06-29 04:40:16 - Warmup-steps: 39


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/389 [00:00<?, ?it/s]

2022-06-29 04:40:51 - CECorrelationEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 200 steps:
2022-06-29 04:40:52 - Correlation:	Pearson: 0.0154	Spearman: -0.0242
2022-06-29 04:40:52 - Save model to output/training_paraphrase_finetuning_training-cross-encoder/stsb-roberta-base-2022-06-29_04-40-10
2022-06-29 04:41:26 - CECorrelationEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2022-06-29 04:41:26 - Correlation:	Pearson: 0.1543	Spearman: 0.1425
2022-06-29 04:41:26 - Save model to output/training_paraphrase_finetuning_training-cross-encoder/stsb-roberta-base-2022-06-29_04-40-10


In [ ]:
import pandas as pd

preds = pd.read_csv('/content/output/Val_results_old_method_thres_0.967.csv')

pairs = []
ids = []
for i,(k,v) in preds.iterrows():
    pairs.append((val_dataset[k], val_dataset[v]))
    ids.append((k,v))

# model_save_path = 'cross-encoder/stsb-roberta-large'
# cross_enc = CrossEncoder(model_save_path)
predictions_confs = cross_enc.predict(pairs)


In [ ]:
predictions_confs[-20:]

array([0.5430016 , 0.80300343, 0.4908086 , 0.5469455 , 0.5933731 ,
       0.84936386, 0.8769619 , 0.67862284, 0.45391378, 0.7520596 ,
       0.657744  , 0.47509566, 0.67427754, 0.68270665, 0.73380136,
       0.73380136, 0.5996247 , 0.54095095, 0.18746984, 0.6291622 ],
      dtype=float32)

In [ ]:
for th in [0.2, 0.5, 0.55, 0.57,  0.59, 0.6, 0.61, 0.62,0.63,0.64, 0.65, 0.66, 0.67,  0.69, 0.7, 0.75, 0.8, 0.85, 0.89, 0.92, 0.95]:
# for th in [0.96, 0.9615, 0.9620, 0.9631, 0.967, 0.968, 0.969, 0.9695]:
    soln_pairs = {}
    for i,p in enumerate(predictions_confs):
        #the id2 filtering is based on similarity scores from first model predictions as the dataframe contained 
        #ordered/sorted pairs with highest similarity first
        if p > th and not ids[i][1] in soln_pairs.values():
            soln_pairs[ids[i][0]] = ids[i][1]
    
    #check if the id2 is present in ids1 keys and if the similarity score is smaller than that pair, include other pair else add current pair
    #basically, if score(id1,id2) > score(id2,id3) include id1,id2 pair else include id2,id3 pair
    new_pairs = {}
    for k in soln_pairs.keys():
        id2 = soln_pairs[k]
        if id2 in soln_pairs.keys() and predictions_confs[[x[0] for x in ids].index(id2)] > predictions_confs[[x[0] for x in ids].index(k)]:
            new_pairs[id2] = soln_pairs[id2]
        else:
            new_pairs[k] = id2

    print(f"percentage retained for {th} is before filtering: ", len(soln_pairs)/len(ids))
    print(f"percentage retained for {th} is after filtering: ", len(new_pairs)/len(ids))
    val_df = pd.DataFrame(new_pairs.items())
    val_df.columns = ['id1','id2']
    df_name = f"output/cross_enc_new_result_{th}.csv"
    val_df.to_csv(df_name,index=False)
    !python /content/evaluation.py --ans '/content/output/cross_enc_new_result_{th}.csv' --evl /content/validation_eval_new.csv
    # !python /content/evaluation.py --ans '/content/output/cross_enc_result_{th}.csv' --evl /content/validation_eval_new.csv


percentage retained for 0.2 is before filtering:  0.8941152092830501
percentage retained for 0.2 is after filtering:  0.8694571073352673
100% 4196/4196 [00:00<00:00, 5461.60it/s]
100% 4196/4196 [00:00<00:00, 7878.96it/s]
Recall: 0.6853507565337001
Precision: 0.474976167778837
F1-score: 0.5610923423423424
percentage retained for 0.5 is before filtering:  0.8622047244094488
percentage retained for 0.5 is after filtering:  0.8416908412764194
100% 4062/4062 [00:00<00:00, 5749.76it/s]
100% 4062/4062 [00:00<00:00, 7847.90it/s]
Recall: 0.6839752407152683
Precision: 0.4896602658788774
F1-score: 0.5707317073170731
percentage retained for 0.55 is before filtering:  0.8435557397430584
percentage retained for 0.55 is after filtering:  0.8251139660174057
100% 3982/3982 [00:00<00:00, 5808.63it/s]
100% 3982/3982 [00:00<00:00, 7606.38it/s]
Recall: 0.6825997248968363
Precision: 0.49849321948769465
F1-score: 0.5761973875181422
percentage retained for 0.57 is before filtering:  0.837546622461666
percenta

**paraphrase mpnet v2 works really great with fine tuned and threshold 0.96 (retained 0.74 recall). If it can be used with cross encoder roberta stb large (pretrained - no fine tune) at threshold 0.64, it will give recall 0.72 and f1 0.576 with total pred around 4400.**
Fine an optimal linking strategy to increase precision. 

**current best is, fine tuned 2 epoch paraphrase mpnet v2, with 0.967 thres gives intermediate results:
```
predicting for 0.967
100% 4659/4659 [00:00<00:00, 4940.48it/s]
100% 4659/4659 [00:00<00:00, 7621.85it/s]
Recall: 0.6960110041265475
Precision: 0.4344279888388066
F1-score: 0.5349544072948328
```
(gives around 4100 values at 0.55 F1 score) 
with pretrained cross encoder at thres 0.64:
```
percentage retained for 0.64 is:  0.8252843957930887
100% 3845/3845 [00:00<00:00, 5883.02it/s]
100% 3845/3845 [00:00<00:00, 7934.03it/s]
Recall: 0.6825997248968363
Precision: 0.5162548764629389
F1-score: 0.587886865096994
```

Fine tuned with new dataset train, paraphrase model gives intermediate results as:
```
predicting for 0.953
100% 5846/5846 [00:01<00:00, 3991.09it/s]
100% 5846/5846 [00:00<00:00, 7399.35it/s]
Recall: 0.7424346629986245
Precision: 0.36931235032500853
F1-score: 0.49326022389764673
predicting for 0.957
100% 5073/5073 [00:01<00:00, 4606.29it/s]
100% 5073/5073 [00:00<00:00, 7589.55it/s]
Recall: 0.7221458046767538
Precision: 0.41395623891188643
F1-score: 0.5262498433780228
predicting for 0.96
100% 4622/4622 [00:00<00:00, 4925.47it/s]
100% 4622/4622 [00:00<00:00, 7590.56it/s]
Recall: 0.7042640990371389
Precision: 0.44309822587624403
F1-score: 0.5439575033200531
predicting for 0.97
100% 3522/3522 [00:00<00:00, 6558.56it/s]
100% 3522/3522 [00:00<00:00, 8010.63it/s]
Recall: 0.6172627235213205
Precision: 0.5096536059057354
F1-score: 0.5583203732503889
predicting for 0.963
100% 4223/4223 [00:00<00:00, 5471.52it/s]
100% 4223/4223 [00:00<00:00, 7793.12it/s]
Recall: 0.6808803301237965
Precision: 0.46886099928960456
F1-score: 0.5553218342448464
```
The new pretrained cross encoder gives results with 0.96 csv pred file:
```
percentage retained for 0.61 is:  0.8788403288619645
100% 4062/4062 [00:00<00:00, 5725.33it/s]
100% 4062/4062 [00:00<00:00, 8020.08it/s]
Recall: 0.6942916093535075
Precision: 0.49704579025110784
F1-score: 0.5793400286944045
percentage retained for 0.62 is:  0.8723496321938555
100% 4032/4032 [00:00<00:00, 5602.52it/s]
100% 4032/4032 [00:00<00:00, 8092.24it/s]
Recall: 0.6911966987620357
Precision: 0.49851190476190477
F1-score: 0.579250720461095
percentage retained for 0.63 is:  0.8641280830809174
100% 3994/3994 [00:00<00:00, 5847.72it/s]
100% 3994/3994 [00:00<00:00, 8067.80it/s]
Recall: 0.6891334250343879
Precision: 0.5017526289434151
F1-score: 0.5807012460156477
percentage retained for 0.64 is:  0.8591518823020338
100% 3971/3971 [00:00<00:00, 5860.01it/s]
100% 3971/3971 [00:00<00:00, 8092.27it/s]
Recall: 0.688445667125172
Precision: 0.5041551246537396
F1-score: 0.5820613461258904
percentage retained for 0.65 is:  0.8496321938554738
100% 3927/3927 [00:00<00:00, 5938.53it/s]
100% 3927/3927 [00:00<00:00, 8093.71it/s]
Recall: 0.6843191196698762
Precision: 0.5067481538069774
F1-score: 0.582297000731529
```

Using same model with new filtering method, it filters out new entries where id 2 are repeated in values for thres 0.96 csv file (with thres 0.957 gives slightly worst results):
```
percentage retained for 0.61 is:  0.8264820424058849
100% 3820/3820 [00:00<00:00, 6132.84it/s]
100% 3820/3820 [00:00<00:00, 8058.53it/s]
Recall: 0.6908528198074277
Precision: 0.5259162303664922
F1-score: 0.597205707491082
percentage retained for 0.62 is:  0.8212894850713977
100% 3796/3796 [00:00<00:00, 6165.81it/s]
100% 3796/3796 [00:00<00:00, 8100.55it/s]
Recall: 0.687757909215956
Precision: 0.5268703898840885
F1-score: 0.5966587112171837
percentage retained for 0.63 is:  0.816313284292514
100% 3773/3773 [00:00<00:00, 6208.30it/s]
100% 3773/3773 [00:00<00:00, 7745.04it/s]
Recall: 0.6860385144429161
Precision: 0.5287569573283859
F1-score: 0.5972159856308935
percentage retained for 0.64 is:  0.8124188662916486
100% 3755/3755 [00:00<00:00, 6244.74it/s]
100% 3755/3755 [00:00<00:00, 8033.73it/s]
Recall: 0.6853507565337001
Precision: 0.5307589880159786
F1-score: 0.5982290259642804
percentage retained for 0.65 is:  0.8052790999567286
100% 3722/3722 [00:00<00:00, 6040.62it/s]
100% 3722/3722 [00:00<00:00, 8093.58it/s]
Recall: 0.6815680880330124
Precision: 0.5325094035464804
F1-score: 0.5978883861236802
```

Again with a new filtering method (simply using no filter method for intermediate results, I moved the (id2 filtering with highest score if id2 in keys) updated filtering method to use after the cross encoder predictions and the results are:
```
percentage retained for 0.61 is before filtering:  0.8264820424058849
percentage retained for 0.61 is after filtering:  0.8197749891821722
100% 3789/3789 [00:00<00:00, 6119.76it/s]
100% 3789/3789 [00:00<00:00, 8108.20it/s]
Recall: 0.6898211829436038
Precision: 0.5294272895223014
F1-score: 0.5990742123338808
percentage retained for 0.62 is before filtering:  0.8212894850713977
percentage retained for 0.62 is after filtering:  0.8147987884032886
100% 3766/3766 [00:00<00:00, 6266.17it/s]
100% 3766/3766 [00:00<00:00, 8074.94it/s]
Recall: 0.686726272352132
Precision: 0.5302708443972385
F1-score: 0.5984417141144741
percentage retained for 0.63 is before filtering:  0.816313284292514
percentage retained for 0.63 is after filtering:  0.8098225876244051
100% 3743/3743 [00:00<00:00, 6251.16it/s]
100% 3743/3743 [00:00<00:00, 8063.16it/s]
Recall: 0.6850068775790922
Precision: 0.5321934277317659
F1-score: 0.5990076680198467
percentage retained for 0.64 is before filtering:  0.8124188662916486
percentage retained for 0.64 is after filtering:  0.8059281696235396
100% 3725/3725 [00:00<00:00, 6222.47it/s]
100% 3725/3725 [00:00<00:00, 8144.36it/s]
Recall: 0.6843191196698762
Precision: 0.5342281879194631
F1-score: 0.6000301522689583
percentage retained for 0.65 is before filtering:  0.8052790999567286
percentage retained for 0.65 is after filtering:  0.7990047598442233
100% 3693/3693 [00:00<00:00, 6179.52it/s]
100% 3693/3693 [00:00<00:00, 8318.92it/s]
Recall: 0.6805364511691885
Precision: 0.5358786894124018
F1-score: 0.5996061202848052
```

In [ ]:
for th in thresholds:
    !echo "predicting for {th}"
    !python /content/evaluation.py --ans '/content/output/cross_enc_results_{th}.csv' --evl /content/validation_eval_new.csv

In [ ]:
x = pd.read_csv("/content/output/cross_enc_result_0.65.csv")
z = x.id2.values.tolist()
from collections import Counter
q = Counter(z)
updated = {}
total = 0
for k,v in q.items():
    if v > 1:
        updated[k] = v
        total += v - 1
updated, total

({}, 0)

Fine tuned 2 epoch paraphrase mpnet v2
predicting for 0.95
100% 13369/13369 [00:07<00:00, 1686.16it/s]
100% 13369/13369 [00:02<00:00, 6479.93it/s]
Recall: 0.7785419532324622
Precision: 0.16934699678360385
F1-score: 0.27818394052958156
predicting for 0.96
100% 6614/6614 [00:01<00:00, 3434.83it/s]
100% 6614/6614 [00:00<00:00, 7043.29it/s]
Recall: 0.7438101788170564
Precision: 0.3270335651648019
F1-score: 0.4543163201008191
predicting for 0.97
100% 4136/4136 [00:00<00:00, 5543.54it/s]
100% 4136/4136 [00:00<00:00, 7653.90it/s]
Recall: 0.671251719394773
Precision: 0.47195357833655704
F1-score: 0.5542305508233958
predicting for 0.963
100% 5604/5604 [00:01<00:00, 4105.41it/s]
100% 5604/5604 [00:00<00:00, 7289.90it/s]
Recall: 0.7273039889958735
Precision: 0.37740899357601715
F1-score: 0.49694548872180455
predicting for 0.965
100% 5086/5086 [00:01<00:00, 4480.97it/s]
100% 5086/5086 [00:00<00:00, 7372.98it/s]
Recall: 0.7125171939477304
Precision: 0.40739284309870233
F1-score: 0.5183887915936953
predicting for 0.967
100% 4659/4659 [00:00<00:00, 4900.12it/s]
100% 4659/4659 [00:00<00:00, 7572.94it/s]
Recall: 0.6960110041265475
Precision: 0.4344279888388066
F1-score: 0.5349544072948328
predicting for 0.98
100% 2957/2957 [00:00<00:00, 7646.20it/s]
100% 2957/2957 [00:00<00:00, 8039.40it/s]
Recall: 0.5299174690508941
Precision: 0.521136286777139
F1-score: 0.5254901960784314
predicting for 0.975
100% 3487/3487 [00:00<00:00, 6580.53it/s]
100% 3487/3487 [00:00<00:00, 7872.17it/s]
Recall: 0.609697386519945
Precision: 0.5084599942644107
F1-score: 0.5544956997654418
predicting for 0.99
100% 1916/1916 [00:00<00:00, 12119.16it/s]
100% 1916/1916 [00:00<00:00, 7602.28it/s]
Recall: 0.3204951856946355
Precision: 0.48643006263048016
F1-score: 0.3864013266998341
predicting for 0.972
100% 3856/3856 [00:00<00:00, 6003.82it/s]
100% 3856/3856 [00:00<00:00, 7707.50it/s]
Recall: 0.6485557083906465
Precision: 0.48910788381742737
F1-score: 0.5576581904198699
predicting for 0.973
100% 3710/3710 [00:00<00:00, 6155.09it/s]
100% 3710/3710 [00:00<00:00, 7895.05it/s]
Recall: 0.6348005502063274
Precision: 0.49757412398921835
F1-score: 0.5578724690238742
predicting for 0.976
100% 3373/3373 [00:00<00:00, 6767.92it/s]
100% 3373/3373 [00:00<00:00, 7917.47it/s]
Recall: 0.5976616231086658
Precision: 0.5152683071449748
F1-score: 0.5534150612959721

fine tune 2 epoch mpnet v1, with updated soln filtering predicting for 0.965
100% 4111/4111 [00:00<00:00, 5346.50it/s]
100% 4111/4111 [00:00<00:00, 7671.88it/s]
Recall: 0.6743466299862448
Precision: 0.47701289224033083
F1-score: 0.5587690554210002
predicting for 0.966
100% 3976/3976 [00:00<00:00, 5903.16it/s]
100% 3976/3976 [00:00<00:00, 7684.10it/s]
Recall: 0.6654057771664375
Precision: 0.48667002012072436
F1-score: 0.5621731551423591
predicting for 0.967
100% 3858/3858 [00:00<00:00, 5910.18it/s]
100% 3858/3858 [00:00<00:00, 7436.82it/s]
Recall: 0.657496561210454
Precision: 0.4955935717988595
F1-score: 0.5651788353532368
predicting for 0.968
100% 3746/3746 [00:00<00:00, 6192.89it/s]
100% 3746/3746 [00:00<00:00, 7991.16it/s]
Recall: 0.6485557083906465
Precision: 0.5034703683929524
F1-score: 0.5668770664262098
predicting for 0.969
100% 3647/3647 [00:00<00:00, 6409.13it/s]
100% 3647/3647 [00:00<00:00, 7817.55it/s]
Recall: 0.6392709766162311
Precision: 0.509734027968193
F1-score: 0.567200610221205
predicting for 0.9695
100% 3603/3603 [00:00<00:00, 6491.19it/s]
100% 3603/3603 [00:00<00:00, 7850.93it/s]
Recall: 0.6330811554332875
Precision: 0.5109630863169581
F1-score: 0.5655045307940407
predicting for 0.97
100% 3559/3559 [00:01<00:00, 3229.93it/s]
100% 3559/3559 [00:00<00:00, 3724.47it/s]
Recall: 0.6262035763411279
Precision: 0.5116605788142736
F1-score: 0.5631668470697387
predicting for 0.9705
100% 3515/3515 [00:00<00:00, 6629.08it/s]
100% 3515/3515 [00:00<00:00, 7927.12it/s]
Recall: 0.6227647867950481
Precision: 0.5152204836415363
F1-score: 0.5639109450412579
predicting for 0.971
100% 3468/3468 [00:00<00:00, 6659.30it/s]
100% 3468/3468 [00:00<00:00, 7944.59it/s]
Recall: 0.6158872077028886
Precision: 0.5164359861591695
F1-score: 0.5617942283563363
predicting for 0.9715
100% 3422/3422 [00:00<00:00, 6786.14it/s]
100% 3422/3422 [00:00<00:00, 7817.20it/s]
Recall: 0.610041265474553
Precision: 0.5184102863822326
F1-score: 0.5605055292259083
predicting for 0.972
100% 3381/3381 [00:00<00:00, 6933.55it/s]
100% 3381/3381 [00:00<00:00, 7945.25it/s]
Recall: 0.6059147180192572
Precision: 0.5211475894705708
F1-score: 0.5603434568293846
predicting for 0.973
100% 3287/3287 [00:00<00:00, 7007.59it/s]
100% 3287/3287 [00:00<00:00, 8111.69it/s]
Recall: 0.5949105914718019
Precision: 0.5263157894736842
F1-score: 0.5585149313962874

# do one test pred submission

In [ ]:
!cp /content/testing_text.rar /content/data/testing_text.rar

cp: cannot stat '/content/testing_text.rar': No such file or directory


In [ ]:
!unrar x -r /content/data/testing_text.rar

In [ ]:
#get the sentences with ids from text

test_root = '/content/text'
test_dataset = BertBaseline.create_dataset(test_root,os.listdir(test_root))
print(len(test_dataset))


test_sol, _ = BertBaseline.get_solution_parapharse_mining(model,test_dataset)

#apply thre of 0.96 and save pred
key_ind_subset = list(test_dataset.keys())
soln_pairs = {}
for (score, id1,id2) in test_sol:
    if 1.0 > score > 0.96 and key_ind_subset[id1] not in soln_pairs.keys():
        soln_pairs[key_ind_subset[id1]] = key_ind_subset[id2]

print(f'total length of soln prediction pairs is:', len(soln_pairs))

save_preds = True
if save_preds:
    val_df = pd.DataFrame(soln_pairs.items())
    val_df.columns = ['id1','id2']
    df_name = f"test_results_new_thres.csv"
    val_df.to_csv(df_name,index=False)

#result with new method, replace pairs where conf is higher for another value, id pair
# key_ind_subset = list(test_dataset.keys())
# print(len(key_ind_subset))
# soln_pairs = {}
# scores_key = {}
# for (score, id1,id2) in test_sol:
#     if 1.0 > score > 0.96:
#         if key_ind_subset[id1] not in soln_pairs.keys():
#             soln_pairs[key_ind_subset[id1]] = [key_ind_subset[id2]]
#             scores_key[key_ind_subset[id1]] = [score]
#         else:
#             soln_pairs[key_ind_subset[id1]].append(key_ind_subset[id2])
#             scores_key[key_ind_subset[id1]].append(score)  
# filtered_pairs = {}
# for k in soln_pairs.keys():
#     filtered_pairs[k] = soln_pairs[k][0]

# lst = []
# new_pairs = {}
# for k in filtered_pairs.keys():
#     id2 = filtered_pairs[k]
#     if id2 in filtered_pairs.keys():
#         if 1.0 > scores_key[id2][0] > scores_key[k][0]:
#             new_pairs[id2] = filtered_pairs[id2]
#             lst.append(id2)
#         else:
#             new_pairs[k] = id2
#     else:
#         new_pairs[k] = id2

# print(f'total length of filtered prediction pairs is:', len(filtered_pairs))


# val_df = pd.DataFrame(filtered_pairs.items())
# val_df.columns = ['id1','id2']
# df_name = f"filtered_with_id2_results.csv"
# val_df.to_csv(df_name,index=False)
# len(lst)

loading training files.......


100%|██████████| 3000/3000 [00:05<00:00, 567.58it/s]


95516


Batches:   0%|          | 0/2985 [00:00<?, ?it/s]

total length of filtered prediction pairs is: 4991


In [ ]:
#now apply another similarity filtering based on cross encoder predictions

preds = pd.read_csv('test_results_new_thres.csv')

pairs = []
ids = []
for i,(k,v) in preds.iterrows():
    pairs.append((test_dataset[k], test_dataset[v]))
    ids.append((k,v))

from sentence_transformers import CrossEncoder
cross_enc = CrossEncoder('cross-encoder/stsb-roberta-large')
predictions_confs = cross_enc.predict(pairs)

print(predictions_confs[10:20])

for th in [0.57, 0.58, 0.59, 0.6, 0.61, 0.62,0.63,0.64, 0.65, 0.66, 0.67, 0.68, 0.7, 0.75, 0.8, 0.85, 0.89, 0.92, 0.95]:
    #new method
    soln_pairs = {}
    for i,p in enumerate(predictions_confs):
        if p > th and not ids[i][1] in soln_pairs.values():
            soln_pairs[ids[i][0]] = ids[i][1]

    #moved filtering from intermediate results to this part based on cross encoder prediction results
    new_pairs = {}
    for k in soln_pairs.keys():
        id2 = soln_pairs[k]
        if id2 in soln_pairs.keys() and predictions_confs[[x[0] for x in ids].index(id2)] > predictions_confs[[x[0] for x in ids].index(k)]:
            new_pairs[id2] = soln_pairs[id2]
        else:
            new_pairs[k] = id2

    print(f"percentage retained for {th} is before filtering: ", len(soln_pairs)/len(ids))
    print(f"percentage retained for {th} is after filtering: ", len(new_pairs)/len(ids))

    print(f"length of final prediction pairs is: ", len(new_pairs))
    val_df = pd.DataFrame(new_pairs.items())
    val_df.columns = ['id1','id2']
    df_name = f"new_preds/cross_enc_result_{th}.csv"
    val_df.to_csv(df_name,index=False)

[0.9705184  0.9728152  0.9731497  0.97450507 0.9742967  0.96790665
 0.97504264 0.97161657 0.96634555 0.9743774 ]
percentage retained for 0.57 is before filtering:  0.8260869565217391
percentage retained for 0.57 is after filtering:  0.8080544980965738
length of final prediction pairs is:  4033
percentage retained for 0.58 is before filtering:  0.8206772189941896
percentage retained for 0.58 is after filtering:  0.8032458425165298
length of final prediction pairs is:  4009
percentage retained for 0.59 is before filtering:  0.814866760168303
percentage retained for 0.59 is after filtering:  0.7980364656381487
length of final prediction pairs is:  3983
percentage retained for 0.6 is before filtering:  0.8098577439390904
percentage retained for 0.6 is after filtering:  0.7934281707072731
length of final prediction pairs is:  3960
percentage retained for 0.61 is before filtering:  0.8038469244640353
percentage retained for 0.61 is after filtering:  0.7876177118813865
length of final predict

In [ ]:
for th in [0.57, 0.58, 0.59, 0.6, 0.61, 0.62,0.63,0.64, 0.65, 0.7, 0.75, 0.8, 0.85, 0.89, 0.92, 0.95]:
    soln_pairs = {}
    for i,p in enumerate(predictions_confs):
        if p > th:
            soln_pairs[ids[i][0]] = ids[i][1]
    print(f"percentage retained for {th} is: ", len(soln_pairs)/len(ids))
    print(f"length of final prediction pairs is: ", len(soln_pairs))
    val_df = pd.DataFrame(soln_pairs.items())
    val_df.columns = ['id1','id2']
    df_name = f"preds/cross_enc_result_{th}.csv"
    val_df.to_csv(df_name,index=False)

percentage retained for 0.57 is:  0.8841541755888651
length of final prediction pairs is:  4129
percentage retained for 0.58 is:  0.8773019271948608
length of final prediction pairs is:  4097
percentage retained for 0.59 is:  0.8710920770877945
length of final prediction pairs is:  4068
percentage retained for 0.6 is:  0.8633832976445396
length of final prediction pairs is:  4032
percentage retained for 0.61 is:  0.8541755888650964
length of final prediction pairs is:  3989
percentage retained for 0.62 is:  0.8458244111349036
length of final prediction pairs is:  3950
percentage retained for 0.63 is:  0.8351177730192719
length of final prediction pairs is:  3900
percentage retained for 0.64 is:  0.8250535331905782
length of final prediction pairs is:  3853
percentage retained for 0.65 is:  0.8177730192719486
length of final prediction pairs is:  3819
percentage retained for 0.7 is:  0.750321199143469
length of final prediction pairs is:  3504
percentage retained for 0.75 is:  0.6494646

Use another strategy which chooses n pairs from a set containing pairs related to two sentence ids (for eg. 1-a, 2-a, 1-b, 4-a etc). Hypothesis is, it should increase precision

In [ ]:
val_soln = pd.read_csv('/content/validation_eval_new.csv')
val_soln.describe()

,id1,id2
count,2908,2908
unique,2908,2908
top,4002_1_3,5670_5_0
freq,1,1


In [ ]:
#for thres score is F1, rec, prec:
# 0.87 | 0.12 | 0.78 | 0.06
# 0.90 | 0.27 | 0.76 | 0.16
# 0.94 | 0.48 | 0.64 | 0.39
# 0.95 | 0.49 | 0.60 | 0.42 
# 0.97 | 0.43 | 0.45 | 0.42



In [ ]:
%%shell
python /content/evaluation.py --ans /content/Val_results_new_thres_0.5.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.7.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.81.csv --evl /content/validation_eval_new.csv
# python /content/evaluation.py --ans /content/Val_results_thres_0.82.csv --evl /content/validation_eval_new.csv
# python /content/evaluation.py --ans /content/Val_results_thres_0.83.csv --evl /content/validation_eval_new.csv
# python /content/evaluation.py --ans /content/Val_results_thres_0.84.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.85.csv --evl /content/validation_eval_new.csv
# python /content/evaluation.py --ans /content/Val_results_thres_0.87.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.88.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.9.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.94.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.95.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.96.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.97.csv --evl /content/validation_eval_new.csv

100% 20010/20010 [00:18<00:00, 1096.23it/s]
100% 20010/20010 [00:03<00:00, 5843.99it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:18<00:00, 1098.00it/s]
100% 20010/20010 [00:03<00:00, 5826.97it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:18<00:00, 1099.80it/s]
100% 20010/20010 [00:03<00:00, 5955.94it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:19<00:00, 1022.73it/s]
100% 20010/20010 [00:03<00:00, 5869.74it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:18<00:00, 1102.04it/s]
100% 20010/20010 [00:03<00:00, 5917.85it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:18<00:00, 1093.95it/s]
100% 20010/20010 [00:03<00:00, 5927.01it/s]
Recall: 0.7771664374140302
Precision:

fine tuned 2 epochs: roberta large
100% 20010/20010 [00:19<00:00, 1018.18it/s]
100% 20010/20010 [00:03<00:00, 5483.03it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:19<00:00, 1017.73it/s]
100% 20010/20010 [00:03<00:00, 5523.31it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:24<00:00, 806.70it/s]
100% 20010/20010 [00:06<00:00, 2954.07it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:21<00:00, 934.19it/s]
100% 20010/20010 [00:03<00:00, 5522.81it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:19<00:00, 1026.24it/s]
100% 20010/20010 [00:03<00:00, 5534.83it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:20<00:00, 977.62it/s]
100% 20010/20010 [00:03<00:00, 5009.87it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:19<00:00, 1027.28it/s]
100% 20010/20010 [00:03<00:00, 5619.87it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:19<00:00, 1030.71it/s]
100% 20010/20010 [00:03<00:00, 5522.76it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:19<00:00, 1021.14it/s]
100% 20010/20010 [00:03<00:00, 5578.22it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:19<00:00, 1028.97it/s]
100% 20010/20010 [00:03<00:00, 5530.68it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 20010/20010 [00:19<00:00, 1028.24it/s]
100% 20010/20010 [00:03<00:00, 5585.49it/s]
Recall: 0.7771664374140302
Precision: 0.11294352823588207
F1-score: 0.1972248887337464
100% 13515/13515 [00:08<00:00, 1528.18it/s]
100% 13515/13515 [00:02<00:00, 5769.78it/s]
Recall: 0.7623796423658872
Precision: 0.16403995560488346
F1-score: 0.2699872130548621
100% 7965/7965 [00:03<00:00, 2567.18it/s]
100% 7965/7965 [00:01<00:00, 5966.45it/s]
Recall: 0.7262723521320495
Precision: 0.26516007532956687
F1-score: 0.388485238664582
100% 5287/5287 [00:01<00:00, 3905.45it/s]
100% 5287/5287 [00:00<00:00, 6131.09it/s]
Recall: 0.640990371389271
Precision: 0.3525628901078116
F1-score: 0.4549115314215985

for pretrained model - roberta large
100% 49215/49215 [02:03<00:00, 398.06it/s]
100% 49215/49215 [00:09<00:00, 5362.63it/s]
Recall: 0.78060522696011
Precision: 0.046124149141521895
F1-score: 0.087101663373175
100% 21280/21280 [00:21<00:00, 973.50it/s]
100% 21280/21280 [00:03<00:00, 5583.66it/s]
Recall: 0.7534387895460798
Precision: 0.10296052631578947
F1-score: 0.18116421365966595
100% 6827/6827 [00:02<00:00, 3037.40it/s]
100% 6827/6827 [00:01<00:00, 6134.57it/s]
Recall: 0.6647180192572214
Precision: 0.2831404716566574
F1-score: 0.3971237801746276
100% 6248/6248 [00:01<00:00, 3262.27it/s]
100% 6248/6248 [00:00<00:00, 6286.70it/s]
Recall: 0.6523383768913342
Precision: 0.3036171574903969
F1-score: 0.41437308868501527
100% 5764/5764 [00:01<00:00, 3565.30it/s]
100% 5764/5764 [00:00<00:00, 6291.08it/s]
Recall: 0.6378954607977991
Precision: 0.32182512144344205
F1-score: 0.42781365313653136
100% 5331/5331 [00:01<00:00, 3872.54it/s]
100% 5331/5331 [00:00<00:00, 6375.31it/s]
Recall: 0.6200137551581844
Precision: 0.3382104670793472
F1-score: 0.4376744750576526
100% 4962/4962 [00:01<00:00, 4159.07it/s]
100% 4962/4962 [00:00<00:00, 6302.32it/s]
Recall: 0.6000687757909215
Precision: 0.3516727126158807
F1-score: 0.44345616264294785
100% 4341/4341 [00:00<00:00, 4675.51it/s]
100% 4341/4341 [00:00<00:00, 6494.99it/s]
Recall: 0.563273727647868
Precision: 0.37733241188666206
F1-score: 0.45192440336598155
100% 4062/4062 [00:00<00:00, 5091.72it/s]
100% 4062/4062 [00:00<00:00, 6484.44it/s]
Recall: 0.5433287482806052
Precision: 0.3889709502708026
F1-score: 0.45337159253945475
100% 3602/3602 [00:00<00:00, 5765.36it/s]
100% 3602/3602 [00:00<00:00, 6399.08it/s]
Recall: 0.49346629986244844
Precision: 0.39838978345363685
F1-score: 0.44086021505376344
100% 2832/2832 [00:00<00:00, 7274.41it/s]
100% 2832/2832 [00:00<00:00, 6651.28it/s]
Recall: 0.37448418156808805
Precision: 0.3845338983050847
F1-score: 0.37944250871080143
100% 2631/2631 [00:00<00:00, 7412.22it/s]
100% 2631/2631 [00:00<00:00, 6613.73it/s]
Recall: 0.3297799174690509
Precision: 0.3645001900418092
F1-score: 0.346271890232894
100% 2430/2430 [00:00<00:00, 8423.95it/s]
100% 2430/2430 [00:00<00:00, 6398.30it/s]
Recall: 0.28370013755158185
Precision: 0.3395061728395062
F1-score: 0.3091045335331585
100% 2240/2240 [00:00<00:00, 8604.59it/s]
100% 2240/2240 [00:00<00:00, 6416.70it/s]
Recall: 0.2389958734525447
Precision: 0.31026785714285715
F1-score: 0.27000777000777004

In [ ]:
%%shell
python /content/evaluation.py --ans /content/Val_results_new_thres_0.975.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.98.csv --evl /content/validation_eval_new.csv
python /content/evaluation.py --ans /content/Val_results_new_thres_0.99.csv --evl /content/validation_eval_new.csv

100% 4434/4434 [00:01<00:00, 2508.14it/s]
100% 4434/4434 [00:01<00:00, 4059.17it/s]
Recall: 0.5797799174690509
Precision: 0.38024357239512857
F1-score: 0.4592754017978752
100% 3771/3771 [00:00<00:00, 5648.09it/s]
100% 3771/3771 [00:00<00:00, 7088.00it/s]
Recall: 0.5140990371389271
Precision: 0.3964465658976399
F1-score: 0.44767180715675997
100% 2627/2627 [00:00<00:00, 8150.10it/s]
100% 2627/2627 [00:00<00:00, 6722.54it/s]
Recall: 0.3156808803301238
Precision: 0.34944803958888465
F1-score: 0.33170731707317075


In [ ]:
!python /content/evaluation.py --ans /content/data/Val_sentenceTransformer_baseline_results_thres_0.81.csv --evl /content/validation_eval_new.csv

100% 8872/8872 [00:03<00:00, 2389.44it/s]
100% 8872/8872 [00:01<00:00, 6271.00it/s]
Recall: 0.6674690508940853
Precision: 0.21877817853922452
F1-score: 0.3295415959252971


## Using TFIDF

In [ ]:
sentences = dataset.values()

from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer object
vectorizer = TfidfVectorizer(stop_words='english')

# Generate matrix of word vectors
tfidf_matrix = vectorizer.fit_transform(sentences)

# Print the shape of tfidf_matrix
print(tfidf_matrix.shape)

(95845, 58675)


In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors=1,
                         metric='cosine',
                         algorithm='brute',
                         n_jobs=-1)
model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=1)

In [ ]:
preds = model.kneighbors()

In [ ]:
sentences_list = list(sentences)
keys = list(dataset.keys())

#get neighbours dist and their corresponding preds
dists, idxs = preds

#threshold for distance filtering
thres = 0

soln_pairs = {}
#we will apply filtering using dists to get correct idxs for each query/sentence
for idx, key in enumerate(keys):
    dist = float(dists[idx])
    pred_key_id = int(idxs[idx])
    #get the sentence key from its corresponding pred index using some distance filter/threshold
    if dist > thres:
        soln_pairs[key] = keys[pred_key_id]
print(len(soln_pairs))

92642


In [ ]:
val_df = pd.DataFrame(soln_pairs.items())
val_df.columns = ['id1','id2']
df_name = f"Val_tfidf_results.csv"
val_df.to_csv(df_name,index=False)

In [ ]:
!python /content/evaluation.py --ans /content/Val_tfidf_results.csv --evl /content/validation_eval_new.csv

 84% 77813/92642 [04:57<00:56, 261.30it/s]
100% 77813/77813 [00:13<00:00, 5944.60it/s]
Recall: 0.7365887207702888
Precision: 0.027527533959621143
F1-score: 0.0530716913814249


In [ ]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

# compute and print the linear kernel matrix
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# compute and print the cosine similarity matrix
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# cosin_sim.shape

In [ ]:
cos_sim = cosine_similarity(tfidf_matrix[0].A.T, tfidf_matrix)
cos_sim

In [ ]:
tfidf_matrix.A

NameError: ignored